## 네이버뉴스 언론사/일자별 동적 웹크롤링

- 실행 전 라이브러리 설치
- pip install webdriver-manager

- 코드 상단 📌 TARGET_URL 수정해서 그대로 실행
- 예시 URL : https://news.naver.com/main/list.naver?mode=LPOD&mid=sec&oid=009&date=20251218
- "oid=009&date=20251218"처럼 날짜 부분까지 나오도록 페이지 확인 후 입력 (언론사 들어가서 바로 나오는 오늘자 페이지는 날짜 표시가 바로 뜨지 않아서, 다른 일자 선택 후 다시 오늘 일자 선택)
- 👉 "oid=009&date=20251218.csv"로 본 파일과 같은 위치에 저장됨. (oid가 언론사 url 코드)

In [4]:
import time
import re
import pandas as pd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# ====================================================
# [크롤링 대상 URL] 📌 여기만 수정!
# ====================================================
TARGET_URL = "https://news.naver.com/main/list.naver?mode=LPOD&mid=sec&oid=009&date=20251219"

# ====================================================
# [보조 함수] 네이버 뉴스 날짜 텍스트를 표준 포맷으로 변환
# ====================================================
def clean_date(date_str):
    """
    입력: '2025.12.19. 오후 2:30' 또는 '기사입력 2025.12.19. 14:30' 등
    출력: '2025-12-19 14:30:00' (datetime 형식의 문자열)
    """
    try:
        # 1. 불필요한 글자 제거 (입력, 수정, 공백 등)
        # 정규식으로 숫자, 점(.), 오전/오후, 시간(:)만 남김
        date_str = date_str.replace("기사입력", "").replace("입력", "").strip()
        
        # 2. 오전/오후 처리 및 변환
        is_pm = "오후" in date_str
        date_str = date_str.replace("오전", "").replace("오후", "").strip()
        
        # 포맷 파싱 (YYYY.MM.DD. H:MM 또는 HH:MM)
        # 네이버는 보통 '2025.12.19. 2:30' 형태로 남음 (오전/오후 제거 후)
        dt = datetime.strptime(date_str, "%Y.%m.%d. %H:%M")
        
        # 오후이고 12시가 아니면 +12시간 / 오전이고 12시면 0시로 변경
        if is_pm and dt.hour != 12:
            dt = dt.replace(hour=dt.hour + 12)
        elif not is_pm and dt.hour == 12: # 오전 12시는 0시
            dt = dt.replace(hour=0)
            
        return dt.strftime("%Y-%m-%d %H:%M:%S")
        
    except Exception as e:
        # 변환 실패 시 (형식이 다르거나 날짜 없음) 원본 반환하되 에러 로그 출력하지 않음(조용히 처리)
        # 필요 시: print(f"날짜 변환 실패: {date_str} -> {e}")
        return date_str

# ====================================================
# 1. 크롬 드라이버 설정
# ====================================================
def set_chrome_driver():
    chrome_options = Options()
    # chrome_options.add_argument("--headless") 
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
    
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    return driver

# ====================================================
# 2. 기사 상세 내용 추출 함수
# ====================================================
def extract_article_info(driver, url):
    try:
        driver.get(url)
        time.sleep(0.5)

        # 연예 기사 필터링
        if "entertain.naver.com" in driver.current_url:
            print(f" -> [패스] 연예 기사")
            return None

        # 제목 로딩 대기
        try:
            WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "#title_area > span"))
            )
        except:
            if "entertain.naver.com" in driver.current_url:
                print(f" -> [패스] 연예 기사(로딩 후 확인)")
                return None
            return None

        # 1) 제목
        try:
            title = driver.find_element(By.CSS_SELECTOR, "#title_area > span").text
        except:
            title = "제목 없음"

        # 2) 본문 (캡션 제거)
        try:
            dic_area = driver.find_element(By.CSS_SELECTOR, "#dic_area")
            driver.execute_script("""
                var element = arguments[0];
                var dirts = element.querySelectorAll(".img_desc, .media_end_summary"); 
                for (var i = 0; i < dirts.length; i++) { dirts[i].remove(); }
            """, dic_area)
            content = dic_area.text.replace("\n", " ").strip()
        except:
            content = "본문 없음"

        # 3) 게재 시간 (추출 후 변환 적용)
        try:
            date_element = driver.find_element(By.CSS_SELECTOR, 
                "#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span"
            )
            raw_date = date_element.text
        except:
            try:
                raw_date = driver.find_element(By.CSS_SELECTOR, ".media_end_head_info_datestamp span").text
            except:
                raw_date = "날짜 없음"
        
        # [수정됨] 날짜 표준화 함수 적용
        clean_date_str = clean_date(raw_date)

        print(f"[수집] {title} ({clean_date_str})")
        
        return {
            "날짜": clean_date_str, 
            "제목": title, 
            "본문": content, 
            "링크": url
        }

    except Exception as e:
        print(f"[에러] {e}")
        return None

# ====================================================
# 3. 메인 실행 함수
# ====================================================
def main():
    driver = set_chrome_driver()
    driver.get(TARGET_URL)
    time.sleep(2)

    collected_data = [] 
    page_count = 1      

    try:
        while True:
            print(f"\n===== [ {page_count} 페이지 ] =====")
            
            # --- URL 수집 ---
            article_urls = []
            selectors = [
                "#main_content > div.list_body.newsflash_body > ul.type06_headline > li dl > dt:not(.photo) > a",
                "#main_content > div.list_body.newsflash_body > ul.type06 > li dl > dt:not(.photo) > a"
            ]

            for sel in selectors:
                links = driver.find_elements(By.CSS_SELECTOR, sel)
                for link in links:
                    url = link.get_attribute("href")
                    if url: article_urls.append(url)
            
            print(f"-> 기사 {len(article_urls)}개 탐색 시작")

            # --- 상세 페이지 순회 ---
            list_window = driver.current_window_handle 
            
            for url in article_urls:
                if "entertain.naver.com" in url: continue

                driver.execute_script("window.open('');")
                driver.switch_to.window(driver.window_handles[-1])
                
                data = extract_article_info(driver, url)
                if data: collected_data.append(data)
                
                driver.close()
                driver.switch_to.window(list_window)
            
            # --- 페이징 ---
            try:
                paging_area = driver.find_element(By.CSS_SELECTOR, "#main_content > div.paging")
                current_page_num = int(paging_area.find_element(By.TAG_NAME, "strong").text)
                next_page_num = current_page_num + 1
                
                next_page_btn = None
                try:
                    next_page_btn = paging_area.find_element(By.XPATH, f".//a[text()='{next_page_num}']")
                except:
                    next_page_btn = None
                
                if next_page_btn:
                    next_page_btn.click()
                    page_count += 1
                    time.sleep(2)
                    continue

                try:
                    driver.find_element(By.CSS_SELECTOR, "a.next").click()
                    page_count += 1
                    time.sleep(2)
                    continue
                except:
                    print("-> 수집 종료 (마지막 페이지)")
                    break

            except:
                break
                
    except KeyboardInterrupt:
        print("\n[중단] 사용자 요청으로 중단합니다.")
    
    finally:
        driver.quit()
        
        # ====================================================
        # 4. 저장 (컬럼 순서 변경 및 파일명 생성)
        # ====================================================
        print("\n[저장] CSV 파일 생성 중...")
        
        if collected_data:
            df = pd.DataFrame(collected_data)
            
            # [수정됨] 컬럼 순서 강제 지정 (날짜 -> 제목 -> 본문 -> 링크)
            df = df[['날짜', '제목', '본문', '링크']]
            
            try:
                if "oid=" in TARGET_URL:
                    params = TARGET_URL.split('?')[1]
                    oid = [p for p in params.split('&') if 'oid=' in p][0]
                    date_val = [p for p in params.split('&') if 'date=' in p][0]
                    filename = f"{oid}&{date_val}.csv"
                else:
                    raise ValueError
            except:
                now = datetime.now().strftime("%Y%m%d_%H%M%S")
                filename = f"naver_news_result_{now}.csv"
            
            df.to_csv(filename, index=False, encoding='utf-8-sig')
            
            print(f"✅ 저장 완료! 파일명: {filename}")
            print(f"📊 총 {len(df)}건 저장됨")
        else:
            print("❌ 수집된 데이터가 없습니다.")

if __name__ == "__main__":
    main()

KeyboardInterrupt: 